In [50]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

file_path="../data/raw/train.csv"
isTrain=True
def AssignedPath(Train):
    if Train==False:
        return False,"../data/raw/test.csv"


isTrain,file_path=AssignedPath(False)
data=pd.read_csv(file_path,delimiter=",")
if isTrain:
    y_train=data["Survived"]
    x_train=data.drop(columns=["Survived"])
else:
    
    x_train=data
print(x_train)

     PassengerId  Pclass                                          Name  \
0            892       3                              Kelly, Mr. James   
1            893       3              Wilkes, Mrs. James (Ellen Needs)   
2            894       2                     Myles, Mr. Thomas Francis   
3            895       3                              Wirz, Mr. Albert   
4            896       3  Hirvonen, Mrs. Alexander (Helga E Lindqvist)   
..           ...     ...                                           ...   
413         1305       3                            Spector, Mr. Woolf   
414         1306       1                  Oliva y Ocana, Dona. Fermina   
415         1307       3                  Saether, Mr. Simon Sivertsen   
416         1308       3                           Ware, Mr. Frederick   
417         1309       3                      Peter, Master. Michael J   

        Sex   Age  SibSp  Parch              Ticket      Fare Cabin Embarked  
0      male  34.5      0      0 

In [51]:
#Filling missing values of age and Embarked

x_train["Age"]=x_train.groupby(["Pclass","Sex"])["Age"].transform(lambda x:x.fillna(x.median()))

x_train["Fare"]=x_train.groupby(["Pclass","Sex"])["Fare"].transform(lambda x:x.fillna(x.median()))

x_train["Embarked"]=x_train["Embarked"].ffill()


In [52]:
#Feature Engineering
x_train["FamilySize"]=x_train["SibSp"]+x_train["Parch"]+1

x_train["isAlone"]=x_train["FamilySize"]==1

#Extracting Titles from names
x_train["Title"]=x_train["Name"].str.extract(r",\s*([^\.]+)\.")
x_train["Title"]=x_train["Title"].replace(["Mlle","Ms"],"Miss")
x_train["Title"]=x_train["Title"].replace(["Mme"],"Mrs")
#Extracting ticket Class or group
x_train["TicketPrefix"]=x_train["Ticket"].str.extract(r"^([A-Za-z./]+)")
x_train["TicketPrefix"]=x_train["TicketPrefix"].fillna("Unknown")

#Extracting Cabin Floor
x_train["CabinLetter"]=x_train["Cabin"].str[0]
x_train["CabinLetter"]=x_train["CabinLetter"].fillna("Unknown")


x_train=x_train.drop(columns=["SibSp","Parch","Name","Ticket","Cabin"])


In [53]:
#One Hot encoding
x_train=pd.get_dummies(x_train,columns=["Pclass","Sex","Embarked","Title","TicketPrefix","CabinLetter"])
print(x_train)

     PassengerId   Age      Fare  FamilySize  isAlone  Pclass_1  Pclass_2  \
0            892  34.5    7.8292           1     True     False     False   
1            893  47.0    7.0000           2    False     False     False   
2            894  62.0    9.6875           1     True     False      True   
3            895  27.0    8.6625           1     True     False     False   
4            896  22.0   12.2875           3    False     False     False   
..           ...   ...       ...         ...      ...       ...       ...   
413         1305  24.0    8.0500           1     True     False     False   
414         1306  39.0  108.9000           1     True      True     False   
415         1307  38.5    7.2500           1     True     False     False   
416         1308  24.0    8.0500           1     True     False     False   
417         1309  24.0   22.3583           3    False     False     False   

     Pclass_3  Sex_female  Sex_male  ...  TicketPrefix_W./C.  \
0        Tr

In [54]:
#Storing the Processed Data
filePath_train="../data/processed/train.csv"
filePath_test="../data/processed/test.csv"

filePath_train_output="../data/processed/train_output.csv"

if isTrain:
    y_train.to_csv(filePath_train_output,index=False)
    x_train.to_csv(filePath_train,index=False)
else:
    x_train.to_csv(filePath_test,index=False)